In [1]:
from clickhouserag.data_access.clickhouse_client import ClickhouseConnectClient

client = ClickhouseConnectClient(
    host="localhost",
    port=9000,
    username="assistantRAG",
    password="safety_password",
    database="assistantdb"
)
client.connect()

In [2]:
table_schema = {
    "id": "UInt32",
    "text": "String",
    "chunk": "String",
    "vector": "Array(Float64)"
}

In [3]:
from clickhouserag.rag.manager import RAGManager

rag_manager = RAGManager(client, "rag_table", table_schema, engine="MergeTree", order_by="id")


In [4]:
from typing import Any, List
import torch
from transformers import AutoModel, AutoTokenizer
from clickhouserag.vectorizers.base import VectorizerBase

class TransformersVectorizer(VectorizerBase):
    """Vectorizer that uses a Transformers model to convert text to vectors."""
    
    def __init__(self, model_name: str) -> None:
        """Initialize the TransformersVectorizer."""
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
    
    def vectorize(self, data: Any) -> List[float]:
        """Convert text data into a vector representation using a Transformers model."""
        if not isinstance(data, str):
            raise ValueError("Data should be a string for text vectorization.")
        
        inputs = self.tokenizer(data, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
            vector = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
        
        return vector

    def bulk_vectorize(self, data: Any) -> List[List[float]]:
        """Convert listed text data into a vector representation using a Transformers model."""

        if not isinstance(data, List[str]):
            raise ValueError("Data should be a list of a strings for text vectorization.")

        inputs = self.tokenizer(
            data, return_tensors="pt", truncation=True, padding=True
        )
        with torch.no_grad():
            outputs = self.model(**inputs)
            vector = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

        return vector

transformers_vectorizer = TransformersVectorizer(model_name="ai-forever/sbert_large_nlu_ru")
rag_manager.add_vectorizer("transformers", transformers_vectorizer)

/home/rebelraider/.cache/pypoetry/virtualenvs/assistant-YmmMni6z-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
datas = [
    {"id": 1, "title": "Нож раскаленный на 1000 градусов против моего друга", "chunk": "Нож раскаленный на 1000 градусов против моего друга"},
    {"id": 2, "title": "Прыжок с крыши на батут — выживет ли?", "chunk": "Прыжок с крыши на батут — выживет ли?"},
    {"id": 3, "title": "Я потратил $10 000 на это! Не повторяйте!", "chunk": "Я потратил $10 000 на это! Не повторяйте!"},
    {"id": 4, "title": "10 способов выжить в пустыне. Это реально!", "chunk": "10 способов выжить в пустыне. Это реально!"},
    {"id": 5, "title": "Попробуй не смеяться: самые смешные моменты!", "chunk": "Попробуй не смеяться: самые смешные моменты!"},
    {"id": 6, "title": "Что произойдет, если не спать 72 часа?", "chunk": "Что произойдет, если не спать 72 часа?"},
    {"id": 7, "title": "Я съел 50 гамбургеров за один час! Реально ли?", "chunk": "Я съел 50 гамбургеров за один час! Реально ли?"},
    {"id": 8, "title": "Секреты миллионеров, которые ты не знал!", "chunk": "Секреты миллионеров, которые ты не знал!"},
    {"id": 9, "title": "Мой друг прыгнул с парашютом... и вот что случилось", "chunk": "Мой друг прыгнул с парашютом... и вот что случилось"},
    {"id": 10, "title": "Разоблачаем фокусы магов: вот как это работает!", "chunk": "Разоблачаем фокусы магов: вот как это работает!"}
]

rag_manager.add_bulk_data(datas, vectorizer_name="transformers")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Failed to insert values into rag_table: 'text'
Failed to add bulk data: Failed to insert values into rag_table


RuntimeError: Failed to add bulk data